In [1]:
import dash_bootstrap_components as dbc
from dash import Dash, Input, Output, html, dcc
from dash_iconify import DashIconify
import dash_mantine_components as dmc

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# import data
kreise_df = pd.read_csv('/home/jan/Uni/DS-Project/modules/dashboard/test/assets/kreise_df.csv')
# rename and drop columns
kreise_df = kreise_df.drop(columns=['Unnamed: 0']).rename(columns={'NAME_1': 'Bundesland', 'NAME_2': 'Region', 'NAME_3': 'Landkreis', 'ENGTYPE_3': 'Land_Stadt'})

In [3]:
def flatten_list(lst):
    flattened_list = []
    for item in lst:
        if type(item) == np.ndarray:
            flattened_list += [x for x in item.tolist()]
        else:
            flattened_list.append(item)
    return flattened_list

def parent_finder(children):
    df = kreise_df
    children_list = df[children].unique().tolist()
    parent_list = []
    parent_column = df.columns.get_loc(children) - 1
    for child in children_list:
        parent_value = df.loc[df[children] == child].iloc[:, parent_column].unique()
        parent_list.append(parent_value)
    return flatten_list(parent_list)

def value_finder(typus):
    df = kreise_df
    value_list = []
    unit_list = df[typus].unique().tolist()
    for unit in unit_list:
        value_value = round(np.mean(df.loc[df[typus] == unit]['avg_Score']), 1)
        value_list.append(value_value)
    return value_list

In [4]:
# create suburst vectors
labels = kreise_df.Bundesland.unique().tolist() + kreise_df.Region.unique().tolist() + kreise_df.Landkreis.unique().tolist()
parents = [''] * len(kreise_df.Bundesland.unique()) + parent_finder('Region') + parent_finder('Landkreis')
values = value_finder('Bundesland') + value_finder('Region') + value_finder('Landkreis')

In [5]:
# create sunburst plot
Ohren_sunburst = px.sunburst(kreise_df, path=['Bundesland', 'Region', 'Landkreis'], values='avg_Score')
Ohren_sunburst.update_traces(insidetextorientation='radial')

In [6]:
fig =go.Figure(go.Sunburst(
    labels=labels[:26],
    parents=parents[:26],
    values=values[:26],
    insidetextorientation='radial'
))
# Update layout for tight margin
fig.update_layout(margin = dict(t=10, l=10, r=10, b=10))

fig.show()

In [7]:
app = Dash(__name__, external_stylesheets = [dbc.themes.LITERA])

In [8]:
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "26rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and add some padding.
CONTENT_STYLE = {
    "margin-left": "28rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

In [11]:
sidebar = html.Div(
    [
        html.H3('Willkommen auf dem Dashboard für PV-Nutzung in "Ohren"'),
        html.Hr(),
        html.P(
            'Erkunden Sie die Potenziale einzelner Ohren', className="lead"
        ),
                # Links und Erklärungen
        dbc.Row([

            dbc.Col(dmc.Button("GitHub Repo",
                                        leftIcon=[DashIconify(icon="bi:github")],
                                        #href="https://github.com/felixschulz385/ds_project",
                                        )),

             dbc.Col(dmc.Button("How it started",
                                        leftIcon=[DashIconify(icon="bi:newspaper")],
                                        #href="https://www.badische-zeitung.de/ob-palmer-will-solaranlagen-auf-freien-stellen-neben-bundesstrassen--211487616.html",
                                        )),
    ])
], style=SIDEBAR_STYLE)

In [12]:
# change to app.layout if running as single page app instead
content = html.Div([
    dbc.Container([
        # Überschriften & Einleitung
        dbc.Row([
            dbc.Col(html.H1("Lustnauer 'Ohren' skaliert auf ganz Deutschland", className="text-center")
                    , className="mb-5 mt-5")
        ]),
        dbc.Row([
            dbc.Col(html.H5(children='Erkennen von ungenutzten Potenzialen in ungenutzten Flächen.'
                                     )
                    , className="mb-4")
            ]),

        dbc.Row([
            dbc.Col(html.H5(children='Auf der zweiten Seite wird eine interaktive Karte dargestellt, welche die verschiedenen Ohren ranked'
                                     'Die dritte Seite präsentiert dieselben Daten in tabellarischer Form, zum erkunden und nachschlagen.')
                    , className="mb-5")
        ]),

        # Sunburst Plot
        dbc.Row([
            dcc.Graph(figure = Ohren_sunburst)
        ])

    ])

], style=CONTENT_STYLE)

In [13]:
app.layout = html.Div([sidebar, content])

# run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [06/Feb/2023 17:00:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:58] "GET /_dash-component-suites/dash_iconify/dash_iconify.v0_1_2m1675698649.min.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:58] "GET /_dash-component-suites/dash_mantine_components/dash_mantine_components.v0_11_1m1675699154.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:58] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:58] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:59] "GET /_dash-component-suites/dash_iconify/async-DashIconify.v0_1_2m1650551946.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:59] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Feb/2023 17:00:59] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


# Test Bereich